### Imports + utility functions

In [28]:
%matplotlib inline

import os
import geopandas as gpd
import numpy as np
import pyproj
from pathlib import Path
from shapely.ops import transform
from shapely.geometry import box
import xarray as xr
import rioxarray as rxr
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from pystac_client import Client

In [3]:
def utm_crs_from_point(lon, lat):
    
    utm_zone = int(np.floor((lon + 180) / 6) + 1)
    
    if lat >= 0:
        utm_crs = 32600 + utm_zone
    else:
        utm_crs = 32700 + utm_zone 

    return utm_crs

## Rationale

While RCM-ARD uses the following polarizations of RR, RL, and RRRL*, some users may want to stick with the circular-linear configuration.

Conversion back to RH/RV/RHRV* to assist with integration of ARD products into a wider range of processes is simple. 

### Work through STAC catalog to find product URLs

In [4]:
# We will search for spring imagery of Gordon Lake, NWT
lon, lat = -113.162, 63.055
utm_crs = utm_crs_from_point(lon, lat)

# Filter assets by imaging date
start_date = '2025-04-01'
end_date = '2025-05-01'

In [5]:
cat_url = 'https://www.eodms-sgdot.nrcan-rncan.gc.ca/stac/'
catalog = Client.open(cat_url)
search = catalog.search(
    collections=['rcm-ard'],
    intersects=dict(type="Point", coordinates=[lon, lat]),
    datetime=f'{start_date}/{end_date}',
    limit=100
)

items = search.item_collection()
print(f'{len(items)} products found')

4 products found


In [6]:
# Get S3 URLs for cloud-optimized GeoTIFFs used in conversion
ard_product = items[0]
order_key = ard_product.properties['order_key']
rl_url, rr_url, rrrl_url = ard_product.assets['rl'].href, ard_product.assets['rr'].href, ard_product.assets['rrrl'].href

In [7]:
rl_url, rr_url, rrrl_url

('https://rcm-ceos-ard.s3.ca-central-1.amazonaws.com/MLC/2025/04/07/RCM2_OK3433412_PK3553866_1_SC30MCPD_20250407_135636_CH_CV_MLC/RCM2_OK3433412_PK3553866_1_SC30MCPD_20250407_135636_RL.tif',
 'https://rcm-ceos-ard.s3.ca-central-1.amazonaws.com/MLC/2025/04/07/RCM2_OK3433412_PK3553866_1_SC30MCPD_20250407_135636_CH_CV_MLC/RCM2_OK3433412_PK3553866_1_SC30MCPD_20250407_135636_RR.tif',
 'https://rcm-ceos-ard.s3.ca-central-1.amazonaws.com/MLC/2025/04/07/RCM2_OK3433412_PK3553866_1_SC30MCPD_20250407_135636_CH_CV_MLC/RCM2_OK3433412_PK3553866_1_SC30MCPD_20250407_135636_RRRL.tif')

#### Read data from COGs using spatial subset

RRRL* GeoTiff contains 2 bands (real + imaginary) so we define our own read function instead of using stackstac, which only supports single-band rasters

In [9]:
def read_cog_from_bbox(cog_url, bbox=None):
    with rxr.open_rasterio(cog_url) as rds:
        # Reproject bounding box to match raster CRS
        if bbox is None:
            out_cog = rds.copy()
        else:
            bbox_geom = box(*bbox)
            cog_epsg = rds.rio.crs.to_epsg()
        
            project = pyproj.Transformer.from_proj(
                pyproj.Proj('epsg:4326'),  # hard-coded in get_bbox() function 
                pyproj.Proj(f'epsg:{cog_epsg}'),
                always_xy = True) 
            
            # Clip COG to reprojected bounds if provided and return
            bbox_geom_proj = transform(project.transform, bbox_geom)
            out_cog = rds.rio.clip_box(*bbox_geom_proj.bounds)
    
    return out_cog

### RR/RL/RRRL* to RH/RV/RHRV* Conversion

#### Step 1: convert original MLC rasters to Stokes parameters

In [10]:
# Option to process entire product extent or subset by bbox
def ard_to_stokes(rl_url, rr_url, rrrl_url, bbox=None):
    rl, rr, rrrl =  read_cog_from_bbox(rl_url, bbox), read_cog_from_bbox(rr_url, bbox), read_cog_from_bbox(rrrl_url, bbox)
    S0 = rl + rr
    S1 = 2 * rrrl[1, :, :]  # 2nd band is imaginary component
    S2 = 2 * rrrl[0, :, :]  # 1st band is real component
    S3 = rl - rr

    return S0[0], S1, S2, S3[0]  # drop redundant "band" dimension when stil present

In [11]:
# processing entire rcm-ard product here - may take a couple minutes to run
S0, S1, S2, S3 = ard_to_stokes(rl_url, rr_url, rrrl_url)

#### Step 2: convert Stokes vectors to RV/RH polarizations

In [12]:
rv = (S0 - S1) / 2
rh = (S0 + S1) / 2

#### Step 3: write results to disk

In [ ]:
# original rasters
out_dir = './data'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

out_rv = f'{out_dir}/{Path(rr_url).stem.replace("_RR", "_RV")}.tif'
rv.rio.to_raster(out_rv)
print(f'Exported RV polarization raster to {out_rv}')

out_rh =f'{out_dir}/{Path(rr_url).stem.replace("_RR", "_RH")}.tif'
rh.rio.to_raster(out_rh)
print(f'Exported RH polarization raster to {out_rh}')

Exported RV polarization raster to ./data/RCM2_OK3433412_PK3553866_1_SC30MCPD_20250407_135636_RV.tif
Exported RH polarization raster to ./data/RCM2_OK3433412_PK3553866_1_SC30MCPD_20250407_135636_RH.tif
Exported RH polarization raster to ./data/RCM2_OK3433412_PK3553866_1_SC30MCPD_20250407_135636_RH.tif


In [38]:
# convert amplitude to decibels (dB) and fix out-of-range values
rv_db = 10 * np.log10(rv)
rv_db = rv_db.where(~np.isneginf(rv_db), np.nan)
rv_db = rv_db.where(~np.isinf(rv_db), np.nan)

rh_db = 10 * np.log10(rh)
rh_db = rh_db.where(~np.isneginf(rh_db), np.nan)
rh_db = rh_db.where(~np.isinf(rh_db), np.nan)

# save to disk
out_rv_db = f'{out_dir}/{Path(rr_url).stem.replace("_RR", "_RV")}_dB.tif'
rv_db.rio.to_raster(out_rv_db)
print(f'Converted RV polarization raster to dB and saved to {out_rv}')

out_rh_db =f'{out_dir}/{Path(rr_url).stem.replace("_RR", "_RH")}_dB.tif'
rh_db.rio.to_raster(out_rh_db)
print(f'Converted RH polarization raster to dB and saved to {out_rh}')

c:\Users\sleipe\AppData\Local\miniconda3\envs\rcm-stac-env\lib\site-packages\xarray\core\computation.py:824: RuntimeWarning: divide by zero encountered in log10
  result_data = func(*input_data)


Converted RV polarization raster to dB and saved to ./data/RCM2_OK3433412_PK3553866_1_SC30MCPD_20250407_135636_RV.tif
Converted RH polarization raster to dB and saved to ./data/RCM2_OK3433412_PK3553866_1_SC30MCPD_20250407_135636_RH.tif
Converted RH polarization raster to dB and saved to ./data/RCM2_OK3433412_PK3553866_1_SC30MCPD_20250407_135636_RH.tif
